# AI 모델 평가: 코드, 인간, 모델 기반 평가

이 노트북에서는 Claude v3와 같은 AI 모델의 효과를 평가하기 위해 널리 사용되는 세 가지 기술에 대해 살펴보겠습니다.

1. 코드 기반 평가
2. 인간 평가
3. 모델 기반 평가

각 접근 방식을 예시를 통해 설명하고 AI 성능을 평가할 때 각각의 장단점을 검토해 보겠습니다.

## 코드 기반 평가 예시: 감정 분석

이 예시에서는 Claude가 영화 리뷰의 감정을 긍정적 또는 부정적으로 분류하는 능력을 평가할 것입니다. 코드를 사용하여 모델의 출력이 예상 감정과 일치하는지 확인할 수 있습니다.

In [ ]:
# Import python's built-in regular expression library
import re

# Import boto3 and json
import boto3
import json
from botocore.exceptions import ClientError
# Store the model name and AWS region for later use

%store -r modelId
%store -r region

In [ ]:
# Function to build the input prompt for sentiment analysis
def build_input_prompt(review):
    user_content = f"""Classify the sentiment of the following movie review as either 'positive' or 'negative' provide only one of those two choices:
    <review>{review}</review>"""
    return [{"role": "user", "content": [{"text": user_content}]}]

# Define the evaluation data
eval = [
    {
        "review": "This movie was amazing! The acting was superb and the plot kept me engaged from start to finish.",
        "golden_answer": "positive"
    },
    {
        "review": "I was thoroughly disappointed by this film. The pacing was slow and the characters were one-dimensional.",
        "golden_answer": "negative"
    }
]

# Function to get completions from the model
bedrock_client = boto3.client(service_name='bedrock-runtime', region_name=region)

def get_completion(messages, system_prompt=None):
    inference_config = {
        "temperature": 0.5,
        "maxTokens": 200
    }
    additional_model_fields = {
        "top_p": 1
    }
    converse_api_params = {
        "modelId": modelId,
        "messages": messages,
        "inferenceConfig": inference_config,
        "additionalModelRequestFields": additional_model_fields
    }
    if system_prompt:
        converse_api_params["system"] = [{"text": system_prompt}]
    try:
        response = bedrock_client.converse(**converse_api_params)
        text_content = response['output']['message']['content'][0]['text']
        return text_content

    except ClientError as err:
        message = err.response['Error']['Message']
        print(f"A client error occured: {message}")

# Get completions for each input
outputs = [get_completion(build_input_prompt(item["review"])) for item in eval]

# Print the outputs and golden answers
for output, question in zip(outputs, eval):
    print(f"Review: {question['review']}\nGolden Answer: {question['golden_answer']}\nOutput: {output}\n")

# Function to grade the completions
def grade_completion(output, golden_answer):
    return output.lower() == golden_answer.lower()

# Grade the completions and print the accuracy
grades = [grade_completion(output, item["golden_answer"]) for output, item in zip(outputs, eval)]
print(f"Accuracy: {sum(grades) / len(grades) * 100}%")

## 인간 평가 예시: 에세이 채점

에세이 채점과 같은 일부 작업은 코드만으로 평가하기 어렵습니다. 이 경우 인간 평가자에게 모델 출력을 평가하기 위한 지침을 제공할 수 있습니다.

In [ ]:
# Function to build the input prompt for essay generation
def build_input_prompt(topic):
    user_content = f"""Write a short essay discussing the following topic:
    <topic>{topic}</topic>"""
    return [{"role": "user", "content": [{"text": user_content}]}]

# Define the evaluation data
eval = [
    {
        "topic": "The importance of education in personal development and societal progress",
        "golden_answer": "A high-scoring essay should have a clear thesis, well-structured paragraphs, and persuasive examples discussing how education contributes to individual growth and broader societal advancement."
    }
]

# Get completions for each input
outputs = [get_completion(build_input_prompt(item["topic"])) for item in eval]

# Print the outputs and golden answers
for output, item in zip(outputs, eval):
    print(f"Topic: {item['topic']}\n\nGrading Rubric:\n {item['golden_answer']}\n\nModel Output:\n{output}\n")

## 모델 기반 평가 예시

Claude에게 모델의 응답과 평가 기준을 제공하여 자체 출력을 평가하도록 할 수 있습니다. 이를 통해 일반적으로 인간의 판단이 필요한 작업을 자동화할 수 있습니다.

### 예시 1: 요약

이 예시에서는 Claude를 사용하여 생성한 요약의 품질을 평가할 것입니다. 모델이 긴 텍스트에서 핵심 정보를 간결하고 정확하게 포착하는 능력을 평가할 때 유용할 수 있습니다. 포함되어야 할 핵심 내용을 개괄한 기준을 제공함으로써 평가 과정을 자동화하고 요약 작업에 대한 모델의 성능을 신속하게 평가할 수 있습니다.

In [ ]:
# Function to build the input prompt for summarization
def build_input_prompt(text):
    user_content = f"""Please summarize the main points of the following text:
    <text>{text}</text>"""
    return [{"role": "user", "content": [{"text": user_content}]}]

# Function to build the grader prompt for assessing summary quality
def build_grader_prompt(output, rubric):
    user_content = f"""Assess the quality of the following summary based on this rubric:
    <rubric>{rubric}</rubric>
    <summary>{output}</summary>
    Provide a score from 1-5, where 1 is poor and 5 is excellent."""
    return [{"role": "user", "content": [{"text": user_content}]}]

# Define the evaluation data
eval = [
    {
        "text": "The Magna Carta, signed in 1215, was a pivotal document in English history. It limited the powers of the monarchy and established the principle that everyone, including the king, was subject to the law. This laid the foundation for constitutional governance and the rule of law in England and influenced legal systems worldwide.",
        "golden_answer": "A high-quality summary should concisely capture the key points: 1) The Magna Carta's significance in English history, 2) Its role in limiting monarchical power, 3) Establishing the principle of rule of law, and 4) Its influence on legal systems around the world."
    }
]

# Get completions for each input
outputs = [get_completion(build_input_prompt(item["text"])) for item in eval]

# Grade the completions
grades = [get_completion(build_grader_prompt(output, item["golden_answer"])) for output, item in zip(outputs, eval)]

# Print the summary quality score
print(f"Summary quality score: {grades[0]}")

### 예시 2: 사실 확인

이 예시에서는 Claude에게 주장을 사실 확인하게 한 다음 그 정확성을 평가할 것입니다. 모델이 정확한 정보와 부정확한 정보를 구분하는 능력을 평가할 때 유용할 수 있습니다. 올바른 사실 확인에 포함되어야 할 핵심 내용을 개괄한 기준을 제공함으로써 평가 과정을 자동화하고 사실 확인 작업에 대한 모델의 성능을 신속하게 평가할 수 있습니다.

In [ ]:
# Function to build the input prompt for fact-checking
def build_input_prompt(claim):
    user_content = f"""Determine if the following claim is true or false:
    <claim>{claim}</claim>"""
    return [{"role": "user", "content": [{"text": user_content}]}]

# Function to build the grader prompt for assessing fact-check accuracy
def build_grader_prompt(output, rubric):
    user_content = f"""Based on the following rubric, assess whether the fact-check is correct:
    <rubric>{rubric}</rubric>
    <fact-check>{output}</fact-check>"""
    return [{"role": "user", "content": [{"text": user_content}]}]

# Define the evaluation data
eval = [
    {
        "claim": "The Great Wall of China is visible from space.",
        "golden_answer": "A correct fact-check should state that this claim is false. While the Great Wall is an impressive structure, it is not visible from space with the naked eye."
    }
]

# Get completions for each input
outputs = [get_completion(build_input_prompt(item["claim"])) for item in eval]

grades = []
for output, item in zip(outputs, eval):
    # Print the claim, fact-check, and rubric
    print(f"Claim: {item['claim']}\n")
    print(f"Fact-check: {output}]\n")
    print(f"Rubric: {item['golden_answer']}\n")
    
    # Grade the fact-check
    grader_prompt = build_grader_prompt(output, item["golden_answer"])
    grade = get_completion(grader_prompt)
    grades.append("correct" in grade.lower())

# Print the fact-checking accuracy
accuracy = sum(grades) / len(grades)
print(f"Fact-checking accuracy: {accuracy * 100}%")

### 예시 3: 톤 분석

이 예시에서는 Claude에게 주어진 텍스트의 톤을 분석하게 한 다음 그 분석의 정확성을 평가할 것입니다. 모델이 텍스트에 표현된 감정적 내용과 태도를 식별하고 해석하는 능력을 평가할 때 유용할 수 있습니다. 식별되어야 할 톤의 핵심 측면을 개괄한 기준을 제공함으로써 평가 과정을 자동화하고 톤 분석 작업에 대한 모델의 성능을 신속하게 평가할 수 있습니다.

In [ ]:
# Function to build the input prompt for tone analysis
def build_input_prompt(text):
    user_content = f"""Analyze the tone of the following text:
    <text>{text}</text>"""
    return [{"role": "user", "content": [{"text": user_content}]}]

# Function to build the grader prompt for assessing tone analysis accuracy
def build_grader_prompt(output, rubric):
    user_content = f"""Assess the accuracy of the following tone analysis based on this rubric:
    <rubric>{rubric}</rubric>
    <tone-analysis>{output}</tone-analysis>"""
    return [{"role": "user", "content": [{"text": user_content}]}]

# Define the evaluation data
eval = [
    {
        "text": "I can't believe they canceled the event at the last minute. This is completely unacceptable and unprofessional!",
        "golden_answer": "The tone analysis should identify the text as expressing frustration, anger, and disappointment. Key words like 'can't believe', 'last minute', 'unacceptable', and 'unprofessional' indicate strong negative emotions."
    }
]

# Get completions for each input
outputs = [get_completion(build_input_prompt(item["text"])) for item in eval]

# Grade the completions
grades = [get_completion(build_grader_prompt(output, item["golden_answer"])) for output, item in zip(outputs, eval)]

# Print the tone analysis quality
print(f"Tone analysis quality: {grades[0]}")

이러한 예시는 코드 기반, 인간 기반, 모델 기반 평가가 Claude와 같은 AI 모델을 다양한 작업에 대해 평가하는 데 사용될 수 있음을 보여줍니다. 평가 방법의 선택은 작업의 성격과 가용 자원에 따라 달라집니다. 모델 기반 평가는 일반적으로 인간의 판단이 필요한 복잡한 작업의 평가를 자동화할 수 있는 유망한 접근 방식입니다.